In [ ]:
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import moola
from dolfin_adjoint import *

In [ ]:
N = 20 # resolution
R = 0.1  # brain circle Radius
ventricle_radius = R/3
aqueduct_width = R/5

brain = Circle(Point(0,0), R)
ventricle = Circle(Point(0,0), ventricle_radius)
aqueduct = Rectangle(Point(-aqueduct_width/2, -R), Point(aqueduct_width/2, 0))

brain = brain - ventricle - aqueduct
mesh = generate_mesh(brain, N)
mesh = Mesh(mesh)

In [ ]:
def solve_darcy(mesh, K, f, dirichlet_boundary_skull=0, dirichlet_boundary_ventricle=0):
    V = FunctionSpace(mesh, "CG", 1)

    u = Function(V)
    K = Constant(K)
    v = TestFunction(V)

    F = ( inner(K*grad(u), grad(v)) - f * v) * dx
    
    ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R)", R = R)
    skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.99 )", R = R)

    bc_ventricle = DirichletBC(V, dirichlet_boundary_ventricle, ventricle, check_midpoint=False)
    bc_skull = DirichletBC(V, dirichlet_boundary_skull, skull, check_midpoint=False)

    bcs = [bc_ventricle, bc_skull]

    solve(F==0, u, bcs)
    return u

In [ ]:
K =1e-4
mmHg2Pa = 133.32

a_cardiac = 1.5 * 7
alpha = 1e-3
control_space = FunctionSpace(mesh, "CG", 1)

p_obs = Expression("(sqrt(x[0]*x[0] + x[1]*x[1]) -R_ventricle) * dpdz * mmHg2Pa",
                   dpdz = a_cardiac, mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                   degree=2)


f = interpolate(Expression("0",  degree=1), control_space)
p = solve_darcy(mesh, K, f, dirichlet_boundary_skull=p_obs, dirichlet_boundary_ventricle=p_obs)
J = assemble( inner(p - p_obs, p - p_obs)*dx + alpha/2 * f**2 * dx )
control = Control(f)
rf = ReducedFunctional(J, control)

problem = MoolaOptimizationProblem(rf)
f_moola = moola.DolfinPrimalVector(f)
solver = moola.BFGS(problem, f_moola, options={'jtol': 0,
                                               'gtol': 1e-9,
                                               'Hinit': "default",
                                               'maxiter': 100,
                                               'mem_lim': 10})
sol = solver.solve()
f_opt = sol['control'].data



In [ ]:
p_opt = solve_darcy(mesh, K, f_opt, dirichlet_boundary_skull=p_obs,dirichlet_boundary_ventricle=p_obs)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,5))
plt.subplot(1,3,1)
c = plot(f_opt, title="f_opt")
plt.colorbar(c)
plt.subplot(1,3,3)
c = plot(p_opt, title="p_opt")
plt.colorbar(c)

plt.subplot(1,3,2)
c = plot(p_obs,mesh=mesh, title="p_obs")
plt.colorbar(c)


In [ ]:
x_vals = np.linspace(0, R, 100)
y = 0
y_vals = np.repeat(y, 100)
in_domain = x_vals**2 + y_vals**2 > ventricle_radius**2

x_in_domain = x_vals[in_domain]
p_vals_init = [p(Point(x,y)) for x in x_in_domain]
p_vals_opt = [p_opt(Point(x,y)) for x in x_in_domain]
f_vals_opt = [f_opt(Point(x,y)) for x in x_in_domain]


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

plt.plot(x_in_domain, p_vals_init, ls="dotted", label="p_initial")
plt.plot(x_in_domain, p_vals_opt, "*",label="p_opt")
plt.ylabel("p in Pa")
plt.legend()
plt.xlabel("x in m")

ax.twinx()
plt.plot(x_in_domain, f_vals_opt, ls="dashed",label="f_opt", color="firebrick")

plt.grid()
plt.ylabel("f in N")
plt.title("$K=${0}".format(K))
plt.legend(loc="lower right")

In [ ]:
# just some random force term for testing...
f_orig = interpolate(Expression("100 *(x[0]+x[1])",  degree=1), control_space)

p_obs = solve_darcy(mesh, K, f_orig, dirichlet_boundary_skull=p_obs, dirichlet_boundary_ventricle=p_obs)

f = interpolate(Expression("0",  degree=1), control_space)
p = solve_darcy(mesh, K, f,dirichlet_boundary_skull=p_obs, dirichlet_boundary_ventricle=p_obs)
J = assemble( inner(p - p_obs, p - p_obs)*dx + alpha/2 * f**2 * dx )
control = Control(f)
rf = ReducedFunctional(J, control)

problem = MoolaOptimizationProblem(rf)
f_moola = moola.DolfinPrimalVector(f)
solver = moola.BFGS(problem, f_moola, options={'jtol': 0,
                                               'gtol': 1e-9,
                                               'Hinit': "default",
                                               'maxiter': 100,
                                               'mem_lim': 10})
sol = solver.solve()
f_opt = sol['control'].data
p_opt = solve_darcy(mesh, K, f_opt, dirichlet_boundary_skull=p_obs,dirichlet_boundary_ventricle=p_obs)

In [ ]:
f_max = np.max([f_opt.vector().max(), f_orig.vector().max()])
f_min = np.max([f_opt.vector().min(), f_orig.vector().min()])

p_max = np.max([p_opt.vector().max(), p_obs.vector().max()])
p_min = np.max([p_opt.vector().min(), p_obs.vector().min()])

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,12))
plt.sca(axes[0][1])
plt.subplot(2,2,1)
plot(f_opt, title="f_opt", vmax = f_max, vmin=f_min)
plt.subplot(2,2,2)
cont = plot(f_orig, title="f_orig", vmax = f_max, vmin=f_min)
plt.subplot(2,2,3)
plot(p_opt, title="p_opt", vmax = p_max, vmin=p_min)
plt.subplot(2,2,4)
cont = plot(p_obs, title="p_obs", vmax = p_max, vmin=p_min)